## Importing Modules

In [3]:
import math
import bokeh
from bokeh.plotting import figure, show
from bokeh.models import tickers, ranges
from bokeh.io import output_notebook
output_notebook()
import numpy as np
import os
import pandas as pd

Loading BokehJS ...

## Setting up for NR

Given

\begin{equation}
\large
y_i = Po({\rm e}^{\theta x_i}) \\
\text{where} \\
z = \Sigma_{i=1}^{n}x_iy_i \hspace{10mm} \lambda_i = {\rm e}^{\theta x_i}
\end{equation}

Before

\begin{equation}
\large
\lambda = a +bx \\
\large
Log \hspace{2 mm} likelihood = L(a,b) = a\hspace{1mm} n \hspace{1mm}\bar y + b\hspace{1mm} z - {\rm e}^{a}\Sigma_{i=1}^{n}{\rm e}^{bx_i}\\
\large
\end{equation}

Now, with $a =0 $

\begin{equation}
\large
b\hspace{1mm} z - (1)\cdot\Sigma_{i=1}^{n}{\rm e}^{bx_i} \\
\\
\large
\text{replace b with $\theta$}
\newline
\newline
\large
\theta\hspace{1mm} z - \Sigma_{i=1}^{n}{\rm e}^{\theta x_i} \\
\large
\theta\hspace{1mm} z - \Sigma_{i=1}^{n}\lambda_i \\
\end{equation}

Derivatives

\begin{equation}
\large
L' (\theta) = \dfrac{\partial L}{\partial \theta} = z - \Sigma_{i=1}^{n}x_i{\rm e}^{\theta x_i} \\
\large
L' (\theta) = \dfrac{\partial L}{\partial \theta} = \Sigma_{i=1}^{n}x_iy_i - \Sigma_{i=1}^{n}x_i{\rm e}^{\theta x_i} \\
\large
L'' (\theta) = \dfrac{\partial^2 L}{\partial\theta^2} = - \Sigma_{i=1}^{n}x_i^2{\rm e}^{\theta x_i}
\end{equation}


Netwon Raphson

\begin{equation}
\large
\theta_{t+1} = \theta_{t} - \frac{L'(x_0)}{L''(x_0)}
\end{equation}

## NR implementation

Given $ n = 10$ and $x_i = -3 + 6i/n$ and $z = 1551$

In [4]:
n = 10
z = 1551
x = np.array([-3 + (6*i)/n for i in range(1,11)])
x    

array([-2.4, -1.8, -1.2, -0.6,  0. ,  0.6,  1.2,  1.8,  2.4,  3. ])

In [5]:
def newton(x, guess, tolerance = 0.00001, output_message = False):
    """
    Args:
        guess (float): initial value for parameter
        tolerance (float): tolerance
    
    \begin{equation}
    t = t_0 - \frac{f(x_0)}{f'(x_0)}.
    \end{equation}

    """
    #Initialize
    t_0 = guess
    difference = tolerance * 5 # Enter Loop
    iter_number = 0

    status_message = 'Starting with Guess = ' + str(t_0) + '\n'
    while abs(difference) > tolerance:
        
        # calculate values of function and derivative and t+1 value
        
        L_prime = z - np.sum(x*np.exp(t_0*x))                
        L_prime2 = - np.sum((x**2)*np.exp(t_0*x))
        t = t_0 - (L_prime / L_prime2)

        # calculate difference and update iteration state
        difference = t - t_0
        t_0 = t
        iter_number += 1
        status_message += 'Iteration #' + str(iter_number) + ':= ' + str(t) + '\n'
        
    status_message += 'Total No. of Iterations = '  +  str(iter_number)
    
    if output_message:
        return t_0, status_message
    return t_0    

In [6]:
# Trial 1
theta_hat = newton(x, guess = 3)
print('Trial 1: Theta_hat =', theta_hat)

# Trial 2
theta_hat = newton(x, guess = -3)
print('Trial 2: Theta_hat =', theta_hat , '\n')

# Trial 3
theta_hat, message = newton(x, guess = 1, output_message = True)
print(message)
print('Trial 3: Theta_hat =', theta_hat)

Trial 1: Theta_hat = 1.9928415099795815
Trial 2: Theta_hat = 1.9928415099736108 

Starting with Guess = 1
Iteration #1:= 6.348995264795304
Iteration #2:= 6.014458020383129
Iteration #3:= 5.679648195038975
Iteration #4:= 5.344504034577839
Iteration #5:= 5.0089522078173685
Iteration #6:= 4.672911152600926
Iteration #7:= 4.336306539727626
Iteration #8:= 3.999121757516986
Iteration #9:= 3.6615462961731198
Iteration #10:= 3.3243917427760756
Iteration #11:= 2.990217388243105
Iteration #12:= 2.6662075099344005
Iteration #13:= 2.3705221112465322
Iteration #14:= 2.140588181045432
Iteration #15:= 2.0202114623084753
Iteration #16:= 1.9938880835642334
Iteration #17:= 1.9928430780678603
Iteration #18:= 1.9928415099771344
Total No. of Iterations = 18
Trial 3: Theta_hat = 1.9928415099771344


In [22]:
t_0  = theta_hat
L_prime = z - np.sum(x*np.exp(theta_hat*x))                
L_prime2 = - np.sum((x**2)*np.exp(theta_hat*x))
print("L' =", L_prime)
print("L'' =", L_prime2)

L' = -1.5419345800182782e-08
L'' = -4376.060653373427


## Variance of Estimator

\begin{equation}
\\
\large
Var \hat\theta = \sigma^2\dfrac{\Sigma_{i=1}^{n}(\dfrac{\partial f(\theta, x_i)}{\partial \theta})^2}{L''(\hat\theta)^2} = \dfrac{Var L'(\theta)}{L''(\hat\theta)^2} = \dfrac{Var \dfrac{\partial L}{\partial \theta}}{L''(\hat\theta)^2} = 
\dfrac{\Sigma_{i=1}^{n}x_i^2\lambda_i}{L''(\hat\theta)^2}
\end{equation}

Estimating Variance $\sigma^2$

In [7]:
print('Theta_hat =', theta_hat)

Theta_hat = 1.9928415099771344


In [9]:
lambda_i =  np.exp(theta_hat*x)
var_L_prime = np.sum(  lambda_i*x**2)
var_L_prime

4376.060653373427

$I''(\hat\theta)$ 

In [16]:
L_prime2 = - np.sum((x**2)*np.exp(theta_hat*x))
L_prime2

-4376.060653373427

$Var(\hat\theta)$ 

In [17]:
var_theta_hat = var_L_prime / (L_prime2)**2
var_theta_hat

0.00022851602827513777

## What is the $p$-value obtained for the test $H_o : \theta = 2$ ? (Use the $\hat\theta$ value rounded to two decimal places)

In [18]:
theta_hat = 1.99

Under Null hypothesis, we would have

$\hat\theta = N(2,Var\hat\theta)$

In [25]:
z_score = abs((theta_hat - 2)) / math.sqrt(var_theta_hat)
z_score

0.6615180007659229

In [26]:
from scipy.stats import norm

In [31]:
p_value = 2 * norm.sf(z_score, loc=0, scale=1)
p_value

0.5082801745412648